In [16]:
from docplex.mp.model import Model

milp_model=Model(name="milp")

T=0
N=0

Parameter

In [17]:
delta = None
gamma=  [[[0 for t in range (T+1)] for j in range(N+1)]  for i in range (N+1)]
gammaDrone= [[0 for j in range (N+1)] for i in range(N+1)]
serving = [0 for i in range (N+1)]
weight = [0 for i in range (N+1)]
cap = 0
energyFull  = 0
beta= 0
gamma =0 

In [18]:
C1=[]
C2=[]
C=[]
K=0
D=0
R=0

Variables

In [19]:
x={}
for i in range (N+2):
    for j in range (N+2):
        for k in range (K+1):
            for t in range (T):
                x[i,j,k,t]=milp_model.binary_var(name=f'x_{i}_{j}_{k}_{t}')

y={}
for i in range (N+2):
    for j in range (N+2):
        for d in range (D+1):
            for r in range (R+1):
                y[i,j,d,r]=milp_model.binary_var(name=f'y_{i}_{j}_{d}_{r}')

timeFinishTruck=milp_model.continuous_var_list(K+1,name="timeFinishTruck")
timeFinishDrone=milp_model.continuous_var_list(D+1,name="timeFinishDrone")
timeWaitTruck=milp_model.continuous_var_list(K+1,name="timeWaitTruck")
timeWaitDrone=milp_model.continuous_var_list(D+1,name="timeWaitDrone")

timeVisitTruck={}
for i in range(N+2):
    for k in range(K+1):
        timeVisitTruck[i,k]=milp_model.continuous_var(name=f'timeVisitTruck_{i}_{k}')

timeVisitDrone={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            timeVisitDrone[i,d,r]=milp_model.continuous_var(name=f'timeVisitDrone_{i}_{d}_{r}')

W={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            W[i,d,r]=milp_model.continuous_var(name=f'W_{i}_{d}_{r}')

energy={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            energy[i,d,r]=milp_model.continuous_var(name=f'energy_{i}_{d}_{r}')

levelTruck={}
for i in range (N+2):
    for k in range (K+1):
        levelTruck[i,k]=milp_model.continuous_var(name=f'levelTruck_{i}_{k}')


levelDrone={}
for i in range (N+2):
    for d in range (D+1):
        for r in range (R+1):
            levelDrone[i,d,r]=milp_model.continuous_var(name=f'levelDrone_{i}_{d}_{r}')

markTruck={}
for i in range (N+2):
    for k in range (K+1):
        markTruck[i,k]=milp_model.continuous_var(name=f'markTruck_{i}_{k}')

markDrone={}
for i in range (N+2):
    for d in range (D+1):
        for r in range (R+1):
            markDrone[i,d,r]=milp_model.continuous_var(name=f'markDrone_{i}_{d}_{r}')

objective

In [23]:
sumFinish=milp_model.max(milp_model.max(timeFinishTruck[k] for k in range (1,K+1)),milp_model.max(timeFinishDrone[d] for d in range (1,D+1)))
sumWait=milp_model.sum(milp_model.sum(timeWaitTruck[k] for k in range (1,K+1))+milp_model.sum(timeWaitDrone[d] for d in range (1,D+1)))

milp_model.set_multi_objective("min",[sumFinish,sumWait])

constraint

In [28]:
#8 if truck leaves depot, must return 
for k in range(1,K+1):
    milp_model.add_constraint(milp_model.sum(x[0,j,k,t] for j in range(1,N+2) for t in range (T+1))
                              ==milp_model.sum(x[i,N+1,k,t] for i in range(0,N+1) for t in range(T+1)))


#9 if truck doesn't leave depot, that truck doesn't move 
for k in range(1,K+1):
     milp_model.add(milp_model.if_then(sum(x[0,j,k,t] for j in range (1,N+2) for t in range (T+1))==0,
                                       sum(x[i,j,k,t] for i in range (0,N+1) for j in range (1,N+2) for t in range (T+1))==0))
     
#11 if drone leaves depot, must return 
for d in range(1,D+1):
    for r in range (1,R+1):
         milp_model.add_constraint(milp_model.sum( y[0,j,d,r]for j in C2 + [N+1] )
                              ==milp_model.sum(y[i,N+1,d,r]for i in [0]+C2))


#12 if drone doesn't leave depot in 1 trip , that drone doesn't move in that trip     

for d in range(1,D+1):
     for r in range (1,R+1):
           milp_model.add(milp_model.if_then(sum(y[0,j,d,r] for j in C2+[N+1])==0,
                                       sum(y[i,j,k,t] for i in [0]+C2 for j in C2+[N+1] )==0))

#14 #15 Ensure no cycle in truck route

for k in range (1,K+1):
     milp_model.add_constraint(levelTruck[0,k]==0)

for k in range (1,K+1):
     for i in range (0,N+1):
          for j in range (1,N+2):
               milp_model.add(milp_model.if_then(sum(x[i,j,k,t] for t in range(T+1))==1,levelTruck[j,k]==levelTruck[i,k]+1))


#16 17 Ensure no cycle in drone route
for D in range (1,D+1):
     for r in range (1,R+1):
         milp_model.add_constraint(levelTruck[0,d,r]==0)

for d in range (1,D+1):
     for r in range (1,R+1):
          for i in [0]+ C2:
               for j in C2+[N+1]:
                   milp_model.add(milp_model.if_then(y[i,j,d,r]==1,levelDrone[j,d,r]==levelDrone[i,d,r]+1))

#18 1 customer that dont require truck only get visited 1 time by truck or drone

for j in C2:
     milp_model.add_constraint(sum(x[i,j,k,t] for i in range(0,N+1) for k in range(1,K+1) for t in range(T+1))+ 
                               sum(y[i,j,d,r] for i in [0]+C2 for d in range (1,D+1) for r in range(1,R+1))==1)

#19 For each customer that require truck, 1 truck need to visit that customer 

for j in C1:
     milp_model.add_constraint(sum(x[i,j,k,t] for i in range(0,N+1) for k in range(1,K+1) for t in range(T+1))==1)

#20  1 customer that dont require truck ,1 truck or drone need to leave that customer

for i in C2:
     milp_model.add_constraint(sum(x[i,j,k,t] for j in range(1,N+2) for k in range(1,K+1) for t in range(T+1))+ 
                               sum(y[i,j,d,r] for j in [N+1]+C2 for d in range (1,D+1) for r in range(1,R+1))==1)

#21 For each customer that require truck, 1 truck need to leave that customer 

for i in C2:
     milp_model.add_constraint(sum(x[i,j,k,t] for j in range(1,N+2) for k in range(1,K+1) for t in range(T+1))==1)

#22 23 If truck/drone visit customer, it must leave that customer

for j in range (1,N+1):
     for k in range (1,K+1):
          milp_model.add_constraint(sum(x[i,j,k,t] for i in range(0,N+1) for t in range(T+1))==
                                    sum(x[j,i,k,t] for i in range(1,N+2) for t in range(T+1)))

for j in C2:
     for d in range(1,D+1):
          for r in range(1,R+1):
               milp_model.add_constraint(sum(y[i,j,d,r] for i in C2+[0])==sum(y[j,i,d,r] for i in C2+[N+1]))


#24 correct sequence of trip

for d in range(1,D+1):
     for r in range(1,R):
          milp_model.add_constraint(sum(y[0,j,d,r] for j in C2)>=sum(y[0,j,d,r+1] for j in C2))


#27

for d in range()


#28


#29




